In [23]:
!pip install networkx


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [24]:
import torch
import torch.nn.functional as F
import networkx as nx
import numpy as np
import torch.optim as optim
from itertools import combinations
import pickle
import cvxpy as cp

In [25]:
# Define a graph
# n = 50  # number of nodes
# m = 613  # number of edges
# G = nx.gnm_random_graph(n, m, seed = 0)

import pickle

#G = nx.gnm_random_graph(n, m, seed = 4) # This graph has a MIS of 5, and it is hard to find!!! They are {0, 2, 9, 10, 11} and {0, 2, 8, 9, 11}
# Create a G(n, m) random graph
def destringizer(value):
    try:
        # Try to convert the string to an integer
        return int(value)
    except ValueError:
        try:
            # Try to convert the string to a float
            return float(value)
        except ValueError:
            # If conversion fails, return the original string
            return value

# Load the .gml graph using nx.read_gml and provide the destringizer function
#G = nx.read_gml("GNM_n50_m613_seed9.gml", destringizer=destringizer)

with open("./graphs/test/GNM_50_613_2.gpickle", "rb") as f:
    G = pickle.load(f)
    

#G = nx.read_gpickle("/content/ER_700_800_0.15_0.gpickle")

# file_path = "/content/uf20-04.gpickle"

# with open(file_path, 'rb') as f:
#     G = pickle.load(f)

n = len(G.nodes)
m = len(G.edges)
#print(G.edges)
complement_G = nx.complement(G)

print(n,m)



### Visualize the graph
#nx.draw(G, with_labels=True, font_weight='bold')

50 613


In [26]:
6
import numpy as np
import networkx as nx
from mosek.fusion import Model, Domain, ObjectiveSense, Matrix, Expr
# Get the adjacency matrix
A = np.array(nx.adjacency_matrix(G).todense())
# Get the number of nodes
n = len(G.nodes)
matrix_of_ones = np.ones((n, n))
# Create a MOSEK model
with Model("SDP_init") as m:
    # Create the matrix variable X
    X = m.variable('X', [n, n], Domain.inPSDCone())
    #ones_mat = mosek.fusion.Expression.constant(1.0, n, n)
    objective_expr = Expr.dot(matrix_of_ones, X)
    # Set the objective: maximize the trace of X
    m.objective('obj', ObjectiveSense.Maximize, objective_expr)
    # Add constraints: trace(X) == 1 and X_{i,j} == 0 for all {i, j} in E
    m.constraint('tr', Expr.sum(X.diag()), Domain.equalsTo(1.0))
    for edge in G.edges:
        i, j = edge
        m.constraint(f'con_{i}_{j}', X.index(i, j), Domain.equalsTo(0.0))
        m.constraint(f'con_{j}_{i}', X.index(j, i), Domain.equalsTo(0.0))
    # Solve the problem
    m.solve()
    # Get the solution
    
    X_solution = X.level()
    X_solution = np.array(X_solution)
    X_solution = X_solution.reshape(n,n)
    #X_solution = m.getPrimalSolution()

# Extract the solution
X_SDP_solution = X_solution

In [27]:
######### Coding up W_SDP_uv for (u,v) not in E, vector s_SDP_v (for the first term from the diagonal of X_SDP), and X_SDP_init:

## Coding W_SDP_uv:
W_SDP_uv = torch.zeros(n,n)

# create a dictionary of key = edge (u,v) in G' and value is equation to X(u,v)
G_hat_edges_strength_dict = {}

for v in range(n):
  for u in range(n):
    if (v,u) not in G.edges() and u != v:
      G_hat_edges_strength_dict[(v,u)] =  X_SDP_solution[v,u]

# Normalize
values_array = np.array(list(G_hat_edges_strength_dict.values()))
normalized_values_MINMAX = (values_array - np.min(values_array)) / (np.max(values_array) - np.min(values_array))
G_hat_edges_strength_dict_normalized_MINMAX = {key: normalized_values_MINMAX[i] for i, key in enumerate(G_hat_edges_strength_dict)} ######## USE THIS

# Assign to W_SDP_uv
for v in range(n):
  for u in range(n):
    if (v,u) not in G.edges() and v != u:
      W_SDP_uv[v,u] = G_hat_edges_strength_dict_normalized_MINMAX[(v,u)]

# print(W_SDP_uv[1,88])
# print(G_hat_edges_strength_dict_normalized_MINMAX[(1,88)])
# print(X_SDP_solution[1,88])

## Coding s_SDP_v and X_SDP_init:

sol_diag = np.diag(X_SDP_solution)
s_SDP_v = sol_diag / np.max(sol_diag)
X_SDP_init = (sol_diag - np.min(sol_diag)) / (np.max(sol_diag) - np.min(sol_diag))
print(len(s_SDP_v))
print(X_SDP_init[0:10])

### Converting to torch tensors: Convert s_SDP_v and W_SDP_uv to torch tensors
s_SDP_v_tensor = torch.tensor(s_SDP_v, dtype=torch.float32)
X_SDP_init_tensor = torch.tensor(X_SDP_init, dtype=torch.float32)
W_SDP_uv_tensor = torch.tensor(W_SDP_uv, dtype=torch.float32)



50
[0.32309889 0.1977664  0.54126133 0.42705276 0.21979998 0.53489048
 0.00790294 0.52872923 0.11137632 0.02165114]


/tmp/ipykernel_42697/129330673.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  W_SDP_uv_tensor = torch.tensor(W_SDP_uv, dtype=torch.float32)


In [28]:
# ### Obtain the A_G matrix

# adjacency_matrix = nx.adjacency_matrix(G)
# adjacency_matrix_dense = adjacency_matrix.todense()
# adjacency_matrix_tensor = torch.tensor(adjacency_matrix_dense, dtype=torch.float32)

# ### Obtain the A_G_hat matrix

# adjacency_matrix_comp = nx.adjacency_matrix(complement_G)
# adjacency_matrix_dense_comp = adjacency_matrix_comp.todense()
# adjacency_matrix_tensor_comp = torch.tensor(adjacency_matrix_dense_comp, dtype=torch.float32)

# def loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma, beta):
#     ## without edges of the comp graph:
#     #loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X)

#     ## with edges of the comp graph:
#     loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp) @ Matrix_X)
#     return loss

# # # test function:
# # Matrix_X = torch.rand((len(G.nodes),1), requires_grad=True)
# # print("testing the function: ", loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma=5, beta = 1))


In [29]:
########## DEFINING THE SDP ENCODED LOSS:

### Obtain the A_G matrix

adjacency_matrix = nx.adjacency_matrix(G)
adjacency_matrix_dense = adjacency_matrix.todense()
adjacency_matrix_tensor = torch.tensor(adjacency_matrix_dense, dtype=torch.float32)

### Obtain the A_G_hat matrix

adjacency_matrix_comp = nx.adjacency_matrix(complement_G)
adjacency_matrix_dense_comp = adjacency_matrix_comp.todense()
adjacency_matrix_tensor_comp = torch.tensor(adjacency_matrix_dense_comp, dtype=torch.float32)



def loss_function_with_SDP(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, s_SDP_v_tensor, W_SDP_uv_tensor, gamma, beta):
    ## without edges of the comp graph:
    #loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X)
    beta = 1

    ## with edges of the comp graph:
    #loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp) @ Matrix_X)
    loss = -(Matrix_X.T @ s_SDP_v_tensor) + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp*W_SDP_uv_tensor) @ Matrix_X)
   
    return loss

# # test function:
# Matrix_X = torch.rand((len(G.nodes),1), requires_grad=True)
# print("testing the function: ", loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma=5, beta = 1))

In [30]:
# Constructing the constant Hessian and looking at the eigen values: This depends on the connectivity of graph... The more edges, the worst, which we already know
gamma = 90 # I select this from the best solution from ReduMIS: Double check the bounds with Rongrong
beta = 1
Hessian = torch.zeros(n, n)
complement_G = nx.complement(G)

Hessian_with_third = (gamma/2)*adjacency_matrix_tensor - (beta/2)*adjacency_matrix_tensor_comp
Hessain_without    = (gamma/2)*adjacency_matrix_tensor

# Compute eigenvalues
eigenvalues_with, _ = torch.linalg.eig(Hessian_with_third)

# Extract the real part of the eigenvalues
real_eigenvalues_with = eigenvalues_with.real


# Compute eigenvalues
eigenvalues_without, _ = torch.linalg.eig(Hessain_without)

# Extract the real part of the eigenvalues
real_eigenvalues_without = eigenvalues_without.real

#print("Eigenvalues without:", real_eigenvalues_without)

# Count positive values
positive_count_with = torch.sum(real_eigenvalues_with > 0).item()

# Count negative values
negative_count_with = torch.sum(real_eigenvalues_with < 0).item()

print("Number of positive values with:", positive_count_with)
print("Number of negative values with:", negative_count_with)

# Count positive values
positive_count_without = torch.sum(real_eigenvalues_without > 0).item()

# Count negative values
negative_count_without = torch.sum(real_eigenvalues_without < 0).item()

print("Number of positive values without:", positive_count_without)
print("Number of negative values without:", negative_count_without)

print(real_eigenvalues_without)



Number of positive values with: 23
Number of negative values with: 27
Number of positive values without: 23
Number of negative values without: 27
tensor([1123.5985,  266.6604,  246.7255,  227.6500,  211.4142,  204.9311,
        -291.5586, -281.4910, -267.3660, -247.1451, -241.7281,  184.6110,
         172.2946,  159.4749,  154.2748, -227.6759,  133.4322,  125.1151,
         113.1092,  110.2599, -212.4972, -206.4571,   94.0410, -194.7748,
        -183.0060,   81.5737, -169.8249, -161.1875, -155.0575, -146.8756,
          72.8810,   63.1747,   49.3723,   41.5685,   31.8052,   17.8051,
           3.5495,  -10.0580,  -14.0165,  -16.7843,  -36.2908,  -55.6776,
        -126.2593,  -69.9073, -111.5998,  -82.0686,  -85.9059, -100.3713,
         -98.0170,  -95.7223])


In [31]:
# This is a function to double check if the MIS we find is a valid IS and/or MIS

def MIS_checker(MIS_list, G):
  pairs = list(combinations(MIS_list, 2))
  IS_CHECKER = True
  if len(MIS_list)>1:
    for pair in pairs:
      if (pair[0], pair[1]) in G.edges or (pair[1], pair[0]) in G.edges:
        IS_CHECKER =  False
        break
  # is it a MIS or IS?
  # if MIS_list is an IS:
  if IS_CHECKER:
    #print("Its an IS")
    # obtain a list of all niebouring nodes
    list_of_all_Ns_in_MIS_list = []
    for node in MIS_list:
      list_of_all_Ns_in_MIS_list.append(list(G.neighbors(node)))

    flattened_list_of_all_Ns_in_MIS_list = [item for sublist in list_of_all_Ns_in_MIS_list for item in sublist]
    #print(flattened_list_of_all_Ns_in_MIS_list)

    list_of_nodes_inG_but_not_in_MIStoCHeck = [item for item in list(G.nodes) if item not in MIS_list]

    MIS_CHECKER = True
    for node in list_of_nodes_inG_but_not_in_MIStoCHeck:
      if node not in flattened_list_of_all_Ns_in_MIS_list:
        MIS_CHECKER = False
        break
  else:
    IS_CHECKER = False
    MIS_CHECKER = False
  return IS_CHECKER, MIS_CHECKER

# test function: use G = nx.gnm_random_graph(15, 52, seed = 4)
print(MIS_checker([0, 1, 2, 3, 4], G))  # Not a valid IS

print(MIS_checker([1, 9], G))  # A valid IS

print(MIS_checker([0, 2, 9, 10, 11], G))  # A valid MIS

print(MIS_checker([0, 2, 9, 10], G))  # A valid IS


print(X_SDP_init_tensor)


(False, False)
(False, False)
(False, False)
(False, False)
tensor([3.2310e-01, 1.9777e-01, 5.4126e-01, 4.2705e-01, 2.1980e-01, 5.3489e-01,
        7.9029e-03, 5.2873e-01, 1.1138e-01, 2.1651e-02, 9.6038e-01, 7.4878e-02,
        6.7653e-01, 8.0132e-02, 6.4952e-02, 9.5737e-01, 5.6174e-01, 6.5433e-01,
        5.4302e-01, 1.7846e-01, 1.0901e-01, 1.6238e-01, 5.2830e-01, 0.0000e+00,
        1.0000e+00, 2.0092e-02, 4.0240e-01, 2.3630e-01, 1.1440e-01, 9.6343e-04,
        2.3926e-03, 7.2893e-01, 3.6792e-02, 2.2788e-01, 2.7230e-01, 2.6553e-01,
        6.8889e-01, 2.5337e-01, 1.7956e-01, 2.2463e-02, 5.1937e-01, 8.1990e-01,
        3.9611e-02, 3.8048e-02, 4.9017e-01, 2.6735e-01, 7.2446e-02, 1.1511e-01,
        5.0320e-01, 5.2167e-01])


In [32]:
start_value = 800
end_value = 100000
step = 800

Indicies_for_complete_restart = list(range(start_value, end_value + 1, step))

print(Indicies_for_complete_restart)

print(X_SDP_init_tensor)

[800, 1600, 2400, 3200, 4000, 4800, 5600, 6400, 7200, 8000, 8800, 9600, 10400, 11200, 12000, 12800, 13600, 14400, 15200, 16000, 16800, 17600, 18400, 19200, 20000, 20800, 21600, 22400, 23200, 24000, 24800, 25600, 26400, 27200, 28000, 28800, 29600, 30400, 31200, 32000, 32800, 33600, 34400, 35200, 36000, 36800, 37600, 38400, 39200, 40000, 40800, 41600, 42400, 43200, 44000, 44800, 45600, 46400, 47200, 48000, 48800, 49600, 50400, 51200, 52000, 52800, 53600, 54400, 55200, 56000, 56800, 57600, 58400, 59200, 60000, 60800, 61600, 62400, 63200, 64000, 64800, 65600, 66400, 67200, 68000, 68800, 69600, 70400, 71200, 72000, 72800, 73600, 74400, 75200, 76000, 76800, 77600, 78400, 79200, 80000, 80800, 81600, 82400, 83200, 84000, 84800, 85600, 86400, 87200, 88000, 88800, 89600, 90400, 91200, 92000, 92800, 93600, 94400, 95200, 96000, 96800, 97600, 98400, 99200, 100000]
tensor([3.2310e-01, 1.9777e-01, 5.4126e-01, 4.2705e-01, 2.1980e-01, 5.3489e-01,
        7.9029e-03, 5.2873e-01, 1.1138e-01, 2.1651e-02, 

In [33]:
# Optimization loop:
# Initialization:
torch.manual_seed(115)
#Matrix_X = torch.zeros((n), requires_grad=True)
#Matrix_X = torch.ones((n), requires_grad=True)
#Matrix_X = torch.rand((n), requires_grad=True)

lower_bound = 0.01
upper_bound = 0.2
# Matrix_X = (upper_bound - lower_bound) * torch.rand(n) + lower_bound
#Matrix_X = 1.0*torch.ones(n)
Matrix_X = X_SDP_init_tensor.clone()
Matrix_X.requires_grad_(True)


# dict for saving... I am thinking of diversification by looking at previous initializations... Still under investigation
dict_of_inits = {}

# This is obtained to get a sense of how far are we from the init
X_ini  = Matrix_X.data.clone()
#gamma = torch.tensor(50.0, requires_grad=True)

learning_rate_alpha = 0.5
number_of_iterations_T = 10000


# Define Optimizer over matrix X
#optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha, weight_decay=10)
optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)
#optimizer = optim.SGD([Matrix_X], lr=learning_rate_alpha)

Best_MIS = 0
for iteration_t in range(number_of_iterations_T):

    loss = loss_function_with_SDP(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X,s_SDP_v_tensor, W_SDP_uv_tensor, gamma = 250, beta = 1)

    optimizer.zero_grad()  # Clear gradients for the next iteration
    loss.backward()  # Backpropagation
    optimizer.step()  # Update the parameters

    # Box-constraining:
    Matrix_X.data[Matrix_X>=1] =1
    Matrix_X.data[Matrix_X<=0] =0


    # Report the current MIS
    MIS = []
    for node in G.nodes:
        if Matrix_X[node] >0.1:
          MIS.append(node)

    # If no MIS, move one
    if MIS_checker(MIS, G)[0] is False: MIS = []

    ## Measuring the gradient sparsity: The results is that the gradient is never sparse
    #sparsity_ratio = ( Matrix_X.grad == 0.0).sum().item() / n
    #print(sparsity_ratio)

    # Iteration logger every XX iterations:
    if (iteration_t + 1) % 200 == 0:
        print(f"Step {iteration_t + 1}/{number_of_iterations_T}, IS: {MIS}, lr: {learning_rate_alpha}, Loss: {loss.item()}")

        #print(f"Step {iteration_t + 1}/{number_of_iterations_T}, IS: {MIS}, lr: {learning_rate_alpha}, Loss: {loss.item()}, grad norm: {torch.norm(Matrix_X.grad).item()}")
        #print(f"Step {iteration_t + 1}/{number_of_iterations_T}, IS: {MIS}, lr: {learning_rate_alpha}, Loss: {loss.item()}")
    if MIS_checker(MIS, G)[1] is True:

      if len(MIS) > Best_MIS: # This is to save our current best so far
        Best_MIS = len(MIS)

      l2_norm = torch.norm(Matrix_X - X_ini, p=2) # If a solution is found, measure its distance from initialization:
      print("+++++++++++ A MIS is found with size: ", [[[len(MIS)]]], "++++ BEST so far",[Best_MIS], "Dist from Init: =",l2_norm.item() , "++++++++++++", [MIS])


    #   #### Local restart every XX iterations:

    #   Matrix_X.data[Matrix_X>0] = 0.8*torch.rand((1))
    #   Matrix_X.data[Matrix_X==0] = 0.2*torch.rand((1))
    #   X_ini = Matrix_X.data.clone()
    #   optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)

    # if iteration_t in Indicies_for_complete_restart:
    #     print("****************&&&&&&&&&&&&&****************&&&&&&&&&&&&& Restarting X Completely at RANDOM &&&&&&&&&&&&&&***************************&&&&&&&&&&&&&")
    #     Matrix_X = (upper_bound - lower_bound) * torch.rand(n) + lower_bound
    #     Matrix_X.requires_grad_(True)
    #     X_ini = Matrix_X.data.clone()
    #     optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)


      # # Restart X and the optimizer to search at a different point in [0,1]^n

      # #lower_bound = 0.2
      # #upper_bound = 1.0
      # Matrix_X = (upper_bound - lower_bound) * torch.rand(n) + lower_bound
      # Matrix_X.requires_grad_(True)


      # #Matrix_X = torch.rand((n), requires_grad=True)
      # X_ini = Matrix_X.data.clone()
      # dict_of_inits[X_ini] = MIS
      # optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)
      # #break

      ##### Restart around the same point that we get from the SDP:
      mean_vector = X_SDP_init_tensor
      covariance_matrix = 1*torch.eye(len(mean_vector))
      Matrix_X = torch.normal(mean=mean_vector, std=torch.sqrt(torch.diag(covariance_matrix)))
      Matrix_X = Matrix_X.detach().clone()
      Matrix_X.data[Matrix_X>=1] =1
      Matrix_X.data[Matrix_X<=0] =0
      Matrix_X.requires_grad_(True)
      optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)
      #optimizer = optim.SGD([Matrix_X], lr=learning_rate_alpha)



# # Visualize the solution:
# node_colors = ['green' if node in MIS else 'blue' for node in G.nodes]
# nx.draw(G, with_labels=True, font_weight='bold', node_color=node_colors)

+++++++++++ A MIS is found with size:  [[[7]]] ++++ BEST so far [7] Dist from Init: = 2.668026924133301 ++++++++++++ [[10, 15, 16, 17, 24, 40, 41]]
Step 200/10000, IS: [31], lr: 0.5, Loss: -0.3208966851234436
+++++++++++ A MIS is found with size:  [[[6]]] ++++ BEST so far [7] Dist from Init: = 2.745237350463867 ++++++++++++ [[12, 16, 24, 31, 40, 41]]
Step 400/10000, IS: [10, 12, 16, 18, 24], lr: 0.5, Loss: -0.9087883830070496
+++++++++++ A MIS is found with size:  [[[6]]] ++++ BEST so far [7] Dist from Init: = 2.814572811126709 ++++++++++++ [[5, 10, 12, 16, 18, 24]]


+++++++++++ A MIS is found with size:  [[[7]]] ++++ BEST so far [7] Dist from Init: = 2.758162260055542 ++++++++++++ [[10, 15, 16, 17, 24, 40, 41]]
Step 600/10000, IS: [], lr: 0.5, Loss: 0.0
+++++++++++ A MIS is found with size:  [[[6]]] ++++ BEST so far [7] Dist from Init: = 2.7861788272857666 ++++++++++++ [[15, 16, 24, 31, 40, 41]]
Step 800/10000, IS: [], lr: 0.5, Loss: -0.3154793381690979
+++++++++++ A MIS is found with size:  [[[6]]] ++++ BEST so far [7] Dist from Init: = 2.836010217666626 ++++++++++++ [[2, 10, 17, 24, 36, 48]]
+++++++++++ A MIS is found with size:  [[[7]]] ++++ BEST so far [7] Dist from Init: = 2.655810594558716 ++++++++++++ [[10, 15, 16, 17, 24, 40, 41]]
Step 1000/10000, IS: [], lr: 0.5, Loss: 0.0
+++++++++++ A MIS is found with size:  [[[7]]] ++++ BEST so far [7] Dist from Init: = 2.7679386138916016 ++++++++++++ [[10, 15, 16, 17, 24, 40, 41]]
Step 1200/10000, IS: [], lr: 0.5, Loss: -0.18531905114650726
+++++++++++ A MIS is found with size:  [[[7]]] ++++ BEST so 